# PUZZLE ONE

In [1]:
import re
import pandas as pd
custs = pd.read_csv('noahs-customers.csv')

In [2]:
custs.head()

,customerid,name,address,citystatezip,birthdate,phone
0,1001,Jack Quinn,201 E Park St,"Los Angeles, CA 91343",1960-05-14,805-287-8515
1,1002,David Powell,224C Tysens Ln,"Staten Island, NY 10306",1978-04-04,516-768-1652
2,1003,Carrie Green,1608 W 53rd Way,"Tampa, FL 33614",1969-01-21,727-209-0470
3,1004,Steven Miller,178½ E 7th St,"Manhattan, NY 10009",1953-08-17,607-941-9563
4,1005,Christine Powers,270 W 242nd St,"Bronx, NY 10463",1983-06-06,212-759-9043


In [3]:
LETTERS_DIGITS = {'ABC':'2', 'DEF':'3', 'GHI':'4', 'JKL':'5',
                'MNO':'6', 'PQRS':'7', 'TUV':'8', 'WXYZ':'9'}

In [4]:
for name0, phone0 in zip(custs['name'], custs['phone']):
    name = re.findall(r' (\w+)$', name0.upper())
    if name:
        phone_match = list(map(lambda x: [LETTERS_DIGITS[k] for k in LETTERS_DIGITS.keys() if x in k][0], name[0])) 
        phone = re.findall(r'\d', phone0)
        if phone == phone_match:
            print(phone0)
            break

488-836-2374


# PUZZLE TWO

In [2]:
products = pd.read_csv('noahs-products.csv')

In [6]:
products.head()

,sku,desc,wholesale_cost
0,DLI0002,Smoked Whitefish Sandwich,9.33
1,PET0005,"Vegan Cat Food, Turkey & Chicken",4.35
2,HOM0018,Power Radio (red),21.81
3,KIT0034,Azure Ladle,2.81
4,PET0041,"Gluten-free Cat Food, Pumpkin & Pumpkin",4.60


In [3]:
orders = pd.read_csv('noahs-orders.csv')

In [13]:
orders.head()

,orderid,customerid,ordered,shipped,items,total
0,1001,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52
1,1002,11683,2017-01-31 00:58:31,2017-01-31 18:00:00,NaN,35.33
2,1003,5676,2017-01-31 01:34:40,2017-01-31 09:00:00,NaN,30.79
3,1004,3097,2017-01-31 02:31:24,2017-01-31 19:45:00,NaN,77.60
4,1005,10374,2017-01-31 02:46:09,2017-01-31 14:45:00,NaN,109.04


In [35]:
orders['shipped']

0         2017-01-31 07:15:00
1         2017-01-31 18:00:00
2         2017-01-31 09:00:00
3         2017-01-31 19:45:00
4         2017-01-31 14:45:00
                 ...         
214202    2022-12-17 16:45:00
214203    2022-12-16 21:25:51
214204    2022-12-18 18:00:00
214205    2022-12-18 08:30:00
214206    2022-12-18 15:30:00
Name: shipped, Length: 214207, dtype: object

In [4]:
order_items = pd.read_csv('noahs-orders_items.csv')

In [15]:
order_items.head()

,orderid,sku,qty,unit_price
0,1001,COL0820,1,25.52
1,1002,TOY8907,1,12.92
2,1002,KIT5813,1,7.99
3,1002,KIT3981,2,7.21
4,1003,KIT7098,1,12.53


In [5]:
big_t = products.merge(order_items, how='inner', on='sku').merge(orders, how='inner', on='orderid',)\
        .merge(custs, how='inner', on='customerid') 

In [42]:
idxs = []
for phone, prod, name, shipped in zip(big_t['phone'], big_t['desc'], big_t['name'], big_t['shipped']):
    if 'rug' in prod.lower() and ''.join(re.findall(r'([A-Z])[a-z]+', name)) == 'JD' and shipped < '2017-06-01':
        print(phone, name)

516-994-3023 Jennifer Daniels
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis


# PUZZLE THREE

In [74]:
custs[custs['name'] == 'Jeremy Davis']
    

,customerid,name,address,citystatezip,birthdate,phone,birthyear,birthday
3163,4164,Jeremy Davis,134-10 Foch Blvd,"South Ozone Park, NY 11420",1995-07-26,212-771-8924,1995,07-26


In [73]:
custs['birthyear'] = custs['birthdate'].apply(lambda x: int(x[:4]))
custs['birthday'] =  custs['birthdate'].apply(lambda x: x[5:])

In [83]:
dog_years = {1922 + 12 * k for k in range(12)}
dog_years = list(map(str, dog_years))
candidates = custs[((custs['birthyear'] - 1982) % 12 == 0) & ('03-20' <= custs['birthday']) & (custs['birthday'] <= '04-20')]
len(candidates)


87

In [88]:
candidates['hood'] = candidates['citystatezip'].apply(lambda x: re.findall(r'^(.+),', x)[0])

C:\Users\etien\AppData\Local\Temp/ipykernel_26500/2512976097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['hood'] = candidates['citystatezip'].apply(lambda x: re.findall(r'^(.+),', x)[0])


In [90]:
candidates[candidates['hood'] == 'South Ozone Park']

,customerid,name,address,citystatezip,birthdate,phone,birthyear,birthday,hood
1273,2274,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397,1958,03-25,South Ozone Park


# PUZZLE FOUR

In [91]:
big_t[big_t['customerid'] == 2274]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone
426464,COL4384,Noah's Poster (white),7.52,123932,1,10.62,2274,2020-06-12 21:19:07,2020-06-14 16:15:00,NaN,24.80,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397
426465,KIT4742,Manual Coffee Bowl,11.27,123932,1,14.18,2274,2020-06-12 21:19:07,2020-06-14 16:15:00,NaN,24.80,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397
426466,HOM8601,Rug Cleaner,3.51,20129,2,4.12,2274,2017-08-12 16:18:23,2017-08-12 16:18:23,NaN,8.24,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397


In [104]:
big_t['pastries?'] = big_t['sku'].apply(lambda x: 'BKY' in x)

In [116]:
big_t['early_shopper'] = big_t['shipped'].apply(lambda x: '04:30:00' < x[-8:] < '05:00:00')

In [117]:
big_t[big_t['early_shopper'] & big_t['pastries?']]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,pastries?,early_shopper
108775,BKY4022,Sesame Bialy,5.86,11996,2,7.40,4640,2017-05-22 00:50:31,2017-05-22 04:45:00,NaN,36.18,Felicia Davis,208½ W 94th St,"Manhattan, NY 10025",1950-05-20,332-718-0838,True,True
127649,BKY0403,Poppyseed Linzer Cookie,4.87,78689,1,5.53,9818,2019-03-18 01:29:09,2019-03-18 04:45:00,NaN,74.21,Glenn Alvarez,110A W 136th St,"Manhattan, NY 10030",1964-03-15,585-729-2248,True,True
144372,BKY5887,Sesame Bagel,6.38,26173,1,7.89,5105,2017-10-11 04:58:38,2017-10-11 04:58:38,NaN,389.65,Heather Brown,2305 Arthur Ave,"Bronx, NY 10458",1970-02-28,929-518-7269,True,True
309844,BKY4940,Raspberry Hamentash,5.31,52609,1,6.76,8078,2018-07-02 01:29:09,2018-07-02 04:45:00,NaN,6.76,Nick Brown,258A W 230th St,"Bronx, NY 10463",1964-02-22,838-347-2025,True,True
367348,BKY2239,Poppyseed Mandelbrot,5.11,96375,1,6.11,2063,2019-09-11 00:56:49,2019-09-11 04:45:00,NaN,6.11,Katie Wilkins,394 W 33rd St,"Manhattan, NY 10001",1968-04-06,838-216-1201,True,True
375720,BKY0455,Raspberry Rugelach,5.24,174534,1,6.36,5375,2021-11-02 04:41:47,2021-11-02 04:41:47,NaN,6.36,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375814,BKY3104,Caraway Puff,4.36,201780,1,5.39,5375,2022-08-03 04:40:13,2022-08-03 04:40:13,NaN,5.39,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375823,BKY4022,Sesame Bialy,5.86,196511,1,7.72,5375,2022-06-11 04:44:42,2022-06-11 04:44:42,NaN,7.72,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375866,BKY7160,Caraway Twist,6.15,199892,2,7.25,5375,2022-07-15 04:38:38,2022-07-15 04:38:38,NaN,14.50,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True


# PUZZLE FIVE

In [18]:
big_t.head(), len(big_t)

(       sku                       desc  wholesale_cost  orderid  qty  \
 0  DLI0002  Smoked Whitefish Sandwich            9.33     1589    1   
 1  TOY8117                Manual Yoyo            4.38     1589    1   
 2  HOM9732             Vintage Widget           14.79     1589    1   
 3  HOM0018          Power Radio (red)           21.81   154309    1   
 4  HOM2248        Mini Mixer (orange)           96.66   154309    1   
 
    unit_price  customerid              ordered              shipped  items  \
 0       10.16       11145  2017-02-05 16:15:23  2017-02-05 20:00:00    NaN   
 1        4.73       11145  2017-02-05 16:15:23  2017-02-05 20:00:00    NaN   
 2       18.58       11145  2017-02-05 16:15:23  2017-02-05 20:00:00    NaN   
 3       27.00       11145  2021-04-12 14:52:46  2021-04-12 16:45:00    NaN   
 4      118.53       11145  2021-04-12 14:52:46  2021-04-12 16:45:00    NaN   
 
     total            name            address     citystatezip   birthdate  \
 0   33.47  

In [20]:
big_t['old_cats'] = big_t['desc'].apply(lambda x: 'cat' in x.lower() and 'senior' in x.lower())

In [21]:
big_t['QV'] = big_t['citystatezip'].apply(lambda x: 'queens village' in x.lower())

In [22]:
target = big_t[big_t['QV'] & big_t['old_cats']]

In [23]:
len(target)

19

In [24]:
target

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,old_cats,QV
400585,PET1220,"Dry Senior Cat Food, Salmon & Shrimp",3.51,181832,1,4.89,5486,2022-01-15 11:14:04,2022-01-15 11:14:04,NaN,115.94,Sam House,220-8 111th Ave,"Queens Village, NY 11429",1955-11-10,607-836-2966,True,True
400588,PET5751,"Dry Senior Cat Food, Beef & Chicken",3.10,181832,1,3.55,5486,2022-01-15 11:14:04,2022-01-15 11:14:04,NaN,115.94,Sam House,220-8 111th Ave,"Queens Village, NY 11429",1955-11-10,607-836-2966,True,True
407942,PET0304,"Gluten-free Senior Cat Food, Chicken & Turkey",4.24,113122,9,5.42,7675,2020-02-25 15:54:18,2020-02-25 20:00:00,NaN,48.78,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407943,PET1849,"Vegan Senior Cat Food, Duck & Pumpkin",4.65,180228,8,5.35,7675,2021-12-30 09:49:55,2021-12-30 19:15:00,NaN,42.80,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407944,PET1909,"Vegan Senior Cat Food, Chicken & Salmon",4.60,140675,10,6.12,7675,2020-11-28 08:19:26,2020-11-28 14:45:00,NaN,61.20,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407945,PET3246,"Dry Senior Cat Food, Chicken & Turkey",3.29,12232,9,3.93,7675,2017-05-24 08:35:47,2017-05-24 18:45:00,NaN,35.37,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407946,PET3395,"Wet Senior Cat Food, Chicken & Chicken",3.02,201100,10,4.05,7675,2022-07-27 14:07:17,2022-07-27 18:30:00,NaN,40.50,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407947,PET4063,"Gluten-free Senior Cat Food, Salmon & Duck",4.40,54605,8,5.94,7675,2018-07-21 13:02:19,2018-07-21 13:02:19,NaN,47.52,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407948,PET4196,"Vegan Senior Cat Food, Salmon & Tuna",4.56,89547,8,6.16,7675,2019-07-03 11:43:17,2019-07-03 17:00:00,NaN,49.28,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True
407949,PET4442,"Dry Senior Cat Food, Tuna & Beef",2.82,9597,10,3.74,7675,2017-04-27 17:58:04,2017-04-27 17:58:04,NaN,37.40,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411,True,True


# PUZZLE SIX

In [25]:
order_items.head()

,orderid,sku,qty,unit_price
0,1001,COL0820,1,25.52
1,1002,TOY8907,1,12.92
2,1002,KIT5813,1,7.99
3,1002,KIT3981,2,7.21
4,1003,KIT7098,1,12.53


In [27]:
mean_prices = order_items.groupby('sku')['unit_price'].agg('mean')

In [35]:
cheap_shoppers = big_t.apply(lambda row: row['unit_price'] < .5 * mean_prices[row['sku']], axis = 1)

In [36]:
big_t[cheap_shoppers]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,old_cats,QV
95704,PET0005,"Vegan Cat Food, Turkey & Chicken",4.35,152309,1,2.20,8342,2021-03-23 14:08:11,2021-03-23 16:15:00,NaN,60.82,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95705,KIT3848,Pearl Pan,10.03,152309,1,5.07,8342,2021-03-23 14:08:11,2021-03-23 16:15:00,NaN,60.82,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95706,KIT5642,Azure Scoop,1.95,152309,1,0.99,8342,2021-03-23 14:08:11,2021-03-23 16:15:00,NaN,60.82,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95707,HOM7008,Power Blender,104.08,152309,1,52.56,8342,2021-03-23 14:08:11,2021-03-23 16:15:00,NaN,60.82,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95708,KIT0100,Super Coconut Fork,2.99,33563,1,1.72,8342,2017-12-24 17:05:45,2017-12-24 17:05:45,NaN,1.72,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95710,KIT9001,Super Steak Ladle,2.94,142363,2,1.67,8342,2020-12-15 09:08:54,2020-12-15 09:08:54,NaN,4.33,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95711,HOM1216,Manual Mixer (orange),94.72,94040,1,53.81,8342,2019-08-18 14:20:28,2019-08-18 14:20:28,NaN,78.95,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95712,KIT4742,Manual Coffee Bowl,11.27,94040,2,6.40,8342,2019-08-18 14:20:28,2019-08-18 14:20:28,NaN,78.95,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95713,HOM5127,Electric Toaster (blue),21.72,94040,1,12.34,8342,2019-08-18 14:20:28,2019-08-18 14:20:28,NaN,78.95,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False
95714,KIT2480,Puce Scale,14.27,154876,1,7.80,8342,2021-04-17 19:52:28,2021-04-17 19:52:28,NaN,76.21,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False


# PUZZLE SEVEN

In [50]:
bool_emily = big_t['name'] == 'Emily Randolph'
bool_has_color = big_t['desc'].apply(lambda x: len(re.findall(r'(\([a-z]+|[A-Z]+\))', x)) != 0)
targets = big_t[bool_has_color & bool_emily]

In [100]:
targets['desc_no_color'] = targets['desc'].apply(lambda x: re.findall(r'.+(?=.\()', x)[0])

C:\Users\etien\AppData\Local\Temp/ipykernel_22244/2065830930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['desc_no_color'] = targets['desc'].apply(lambda x: re.findall(r'.+(?=.\()', x)[0])


In [108]:
big_t_colors = big_t[bool_has_color]
big_t_colors['desc_no_color'] = big_t_colors['desc'].apply(lambda x: re.findall(r'.+(?=.\()', x)[0])

C:\Users\etien\AppData\Local\Temp/ipykernel_22244/728073287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_t_colors['desc_no_color'] = big_t_colors['desc'].apply(lambda x: re.findall(r'.+(?=.\()', x)[0])


In [119]:
big_t_colors

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,old_cats,QV,desc_no_color
3,HOM0018,Power Radio (red),21.81,154309,1,27.00,11145,2021-04-12 14:52:46,2021-04-12 16:45:00,NaN,149.06,Frederick Moss,2855 Bronx Park E,"Bronx, NY 10467",1971-07-15,917-807-7174,False,False,Power Radio
4,HOM2248,Mini Mixer (orange),96.66,154309,1,118.53,11145,2021-04-12 14:52:46,2021-04-12 16:45:00,NaN,149.06,Frederick Moss,2855 Bronx Park E,"Bronx, NY 10467",1971-07-15,917-807-7174,False,False,Mini Mixer
9,COL3822,Noah's Jersey (blue),18.56,163994,1,24.50,11145,2021-07-18 20:18:05,2021-07-18 20:18:05,NaN,39.60,Frederick Moss,2855 Bronx Park E,"Bronx, NY 10467",1971-07-15,917-807-7174,False,False,Noah's Jersey
20,HOM2857,Electric Bicycle (orange),79.33,159813,1,102.97,11145,2021-06-07 11:18:37,2021-06-07 11:18:37,NaN,107.21,Frederick Moss,2855 Bronx Park E,"Bronx, NY 10467",1971-07-15,917-807-7174,False,False,Electric Bicycle
22,COL1909,Noah's Gift Box (white),5.48,132884,1,6.04,11145,2020-09-10 01:59:56,2020-09-10 04:30:00,NaN,6.04,Frederick Moss,2855 Bronx Park E,"Bronx, NY 10467",1971-07-15,917-807-7174,False,False,Noah's Gift Box
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427767,HOM9772,Automatic Blender (mauve),46.52,78045,1,60.79,4598,2019-03-11 16:22:52,2019-03-12 18:00:00,NaN,60.79,Brooke Stewart,2924 S 7th Pl,"Tallahassee, FL 32399",1964-08-15,813-871-1275,False,False,Automatic Blender
427768,COL9842,Noah's Gift Box (orange),5.66,57745,1,7.06,11418,2018-08-22 13:19:40,2018-08-22 14:30:00,NaN,7.06,Phillip Coleman,17 Pyramid Ct,"Staten Island, NY 10314",1956-10-27,838-636-7425,False,False,Noah's Gift Box
427769,HOM9868,Mini Radio (white),18.24,22492,1,24.55,1904,2017-09-05 00:30:38,2017-09-05 00:30:38,NaN,24.55,John Williams,14-77 165th St,"Whitestone, NY 11357",1957-12-29,315-305-2177,False,False,Mini Radio
427770,HOM9868,Mini Radio (white),18.24,113225,1,22.76,2739,2020-02-26 16:54:16,2020-02-26 16:54:16,NaN,22.76,James Clarke,122 Schaefer St,"Brooklyn, NY 11207",1990-04-24,212-654-3999,False,False,Mini Radio


In [120]:
days = targets['ordered'].unique()

In [155]:
days[0][:15]

'2019-08-18 14:2'

In [156]:
same_days = [big_t_colors[big_t_colors['ordered'].apply(lambda x: x[:15] == day[:15])] 
for day in days]

In [157]:
[len(df) for df in same_days]

[2, 2, 1, 1, 4, 2]

In [158]:
same_days[0].head()

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,old_cats,QV,desc_no_color
95711,HOM1216,Manual Mixer (orange),94.72,94040,1,53.81,8342,2019-08-18 14:20:28,2019-08-18 14:20:28,NaN,78.95,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False,Manual Mixer
95713,HOM5127,Electric Toaster (blue),21.72,94040,1,12.34,8342,2019-08-18 14:20:28,2019-08-18 14:20:28,NaN,78.95,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False,Electric Toaster


In [159]:
items_em = [targets['desc_no_color'][targets['ordered'].apply(lambda x : x == day)] for day in days]

In [160]:
items_em[0]

95711        Manual Mixer
95713    Electric Toaster
Name: desc_no_color, dtype: object

In [161]:
same_items = [df[df['desc_no_color'].apply(lambda x : x in items.values)] for df, items in zip(same_days, items_em)]

In [162]:
[len(df) for df in same_items]

[2, 2, 1, 1, 2, 1]

In [165]:
same_items[-2]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,old_cats,QV,desc_no_color
95722,HOM5139,Electric Machine (amber),31.81,86371,1,15.90,8342,2019-06-01 12:50:16,2019-06-01 12:50:16,NaN,15.90,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False,False,Electric Machine
425037,HOM7757,Electric Machine (purple),29.43,86372,1,34.82,8835,2019-06-01 12:50:25,2019-06-01 12:50:25,NaN,34.82,Jonathan Adams,644 Targee St,"Staten Island, NY 10304",1975-08-26,315-618-5263,False,False,Electric Machine


# PUZZLE EIGHT

In [166]:
products

,sku,desc,wholesale_cost
0,DLI0002,Smoked Whitefish Sandwich,9.33
1,PET0005,"Vegan Cat Food, Turkey & Chicken",4.35
2,HOM0018,Power Radio (red),21.81
3,KIT0034,Azure Ladle,2.81
4,PET0041,"Gluten-free Cat Food, Pumpkin & Pumpkin",4.60
...,...,...,...
1119,TOY9939,Transformers Board Game,17.54
1120,PET9974,"Vegan Senior Cat Food, Salmon & Pumpkin",5.00
1121,PET9988,"Gluten-free Senior Cat Food, Chicken & Beef",4.32
1122,HOM9988,Mini Crockpot (purple),59.93


In [172]:
noahs_collectibles = products[products['sku'].apply(lambda x : x.startswith('COL'))]

In [173]:
noahs_collectibles

,sku,desc,wholesale_cost
30,COL0263,Noah's Jersey (orange),21.08
38,COL0295,Noah's Jersey (red),15.42
53,COL0464,Noah's Action Figure (magenta),23.93
68,COL0638,Noah's Jewelry (purple),43.98
76,COL0732,Noah's Gift Box (mauve),5.01
...,...,...,...
1071,COL9448,Noah's Poster (blue),7.60
1077,COL9469,Noah's Jewelry (puce),64.02
1091,COL9688,Noah's Bobblehead (azure),11.42
1092,COL9689,Noah's Jewelry (azure),62.01


In [178]:
collectors = big_t[big_t['sku'].apply(lambda x: x.startswith('COL'))].groupby(by='customerid')['sku'].agg('nunique')

In [181]:
collectors[collectors == 81]

customerid
4308    81
Name: sku, dtype: int64

In [182]:
custs[custs['customerid'] == 4308]

,customerid,name,address,citystatezip,birthdate,phone
3307,4308,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980
10499,4308,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980
